<a href="https://colab.research.google.com/github/sanketrk/Clustering-of-Network-Complaint-Data/blob/master/2_Hands_on_Using_ChatGPT_with_Python_and_LangChain_for_real_world_tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using ChatGPT with Python and LangChain for real-world tasks

In this notebook you will use ChatGPT and LangChain to solve and learn about:

- Langchain chains
- Memory and conversation chains

- Exercise 1: Review Analysis and Response
- Exercise 2: Paper Analysis and Summarization


Bonus: Build a text-based chatbot

___[Created By: Dipanjan (DJ)](https://www.linkedin.com/in/dipanjans/)___

## Install OpenAI and LangChain dependencies

In [ ]:
!pip install openai
!pip install langchain langchain-openai

## Load OpenAI API Credentials

Here we load it from a file so we don't explore the credentials on the internet by mistake

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
import yaml

with open('chatgpt_api_credentials.yml', 'r') as file:
    api_creds = yaml.safe_load(file)

In [ ]:
api_creds.keys()

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = api_creds['openai_key']

## Load Necessary Dependencies and LLM

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [ ]:
model = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.0)

## Let's look at how to create a basic chain

In [ ]:
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
chain = (
         prompt
         |
         model
)

response = chain.invoke({"topic": "bears"})
print(response.content)

In [ ]:
# can be used on multiple prompts also
topics = [{'topic': 'AI'}, {'topic': 'Statistics'}]
responses = chain.map().invoke(topics)

In [ ]:
for response in responses:
  print(response.content)
  print('-----')
  print('\n')

## Basic chains are ad-hoc - No conversation history!

In [ ]:
prompt = ChatPromptTemplate.from_template("{query}")
basic_chain = (
               prompt
               |
              model
)

In [ ]:
response = basic_chain.invoke({"query" : 'What are the first four colors of the rainbow?'})
print(response.content)

In [ ]:
response = basic_chain.invoke({"query" : 'And the other three?'})
print(response.content) # gives a totally random response

## Let's learn how to add memory to build a conversation chain

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [ ]:
# create prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Act as a helpful AI Assistant"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

memory = ConversationBufferMemory(return_messages=True)

In [ ]:
memory.load_memory_variables({}) # shows the conversation history

In [ ]:
from operator import itemgetter
# creates the conversation chain
chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables)
        |
        itemgetter("history")
    )
    |
    prompt
    |
    model
)

In [ ]:
user_input = {'input': 'What are the first four colors of a rainbow'}
response = chain.invoke(user_input)
response.content

In [ ]:
memory.save_context(user_input, {"output": response.content})
memory.load_memory_variables({}) # remembers the conversation history

In [ ]:
user_input = {'input': 'And the last 3?'}
response = chain.invoke(user_input) # uses history of the past conversation to give a better response
response.content

In [ ]:
memory.save_context(user_input, {"output": response.content})
memory.load_memory_variables({})

## Exercise 1: Review Analysis and Response

For each review get ChatGPT to do the following and use LLMChain:

  - Summarize the review below, delimited by triple
  backticks. The summary should be at most 3 lines.
  - Highlight both the positives and negatives
  - Display the overall sentiment of the review (positive, negative, neutral)
  - Display a list of 3 - 5 emotions expressed by the customer in the review
  - If the sentiment is positive or neutral write an email and thank them for the review
  - If the sentiment is negative apologize and write an email with an appropriate response


Remember you can pass the list of documents using the `map().invoke(...)` function with your `chain`

In [ ]:
reviews = [
    f"""
    Just received the Bluetooth speaker I ordered for beach outings, and it's fantastic.
    The sound quality is impressively clear with just the right amount of bass.
    It's also waterproof, which tested true during a recent splashing incident.
    Though it's compact, the volume can really fill the space.
    The price was a bargain for such high-quality sound.
    Shipping was also on point, arriving two days early in secure packaging.
    """,
    f"""
    Purchased a new gaming keyboard because of its rave reviews about responsiveness and backlighting.
    It hasn't disappointed. The keys have a satisfying click and the LED colors are vibrant,
    enhancing my gaming experience significantly. Price-wise, it's quite competitive,
    and I feel like I got a good deal. The delivery was swift, and it came well-protected,
    ensuring no damage during transport.
    """,
    f"""
    Ordered a set of wireless earbuds for running, and they've been a letdown.
    The sound constantly cuts out, and the fit is uncomfortable after only a few minutes of use.
    They advertised a 12-hour battery life, but I'm barely getting four hours.
    Considering the cost, I expected better quality and performance.
    They did arrive on time, but the positives end there. I'm already looking into a return.
    """,
    f"""
    The tablet stand I bought was touted as being sturdy and adjustable,
    but it's anything but. It wobbles with the slightest touch,
    and the angles are not holding up as promised. It feels like a breeze could knock it over.
    It was also pricier than others I've seen, which adds to the disappointment.
    It did arrive promptly, but what's the use if the product doesn't meet basic expectations?
    """,
    f"""
    Needed a new kitchen blender, but this model has been a nightmare.
    It's supposed to handle various foods, but it struggles with anything tougher than cooked vegetables.
    It's also incredibly noisy, and the 'easy-clean' feature is a joke; food gets stuck under the blades constantly.
    I thought the brand meant quality, but this product has proven me wrong.
    Plus, it arrived three days late. Definitely not worth the expense.
    """
]

In [ ]:
prompt = """
            Act as a product review analyst.
            Your task is to perform the following tasks:

            - Summarize the review below, delimited by triple
            backticks. The summary should be at most 3 lines.
            - Highlight both the positives and negatives
            - Display the overall sentiment of the review (positive, negative, neutral)
            - Display a list of 3 - 5 emotions expressed by the customer in the review
            - If the sentiment is positive or neutral write an email and thank them for the review
            - If the sentiment is negative apologize and write an email with an appropriate response

            ```{review}```
"""

reviews_formatted =
prompt_template =
llm_chain =

In [ ]:
results =

In [ ]:
for result in results:
  print(result.content)
  print('-----')
  print('\n')

## Exercise 2: Paper Analysis and Summarization

- Act as a Artificial Intelligence Expert.
Transform this research paper abstract in triple backticks
into a short concise version of maximum 10 lines for your audience.

- Act as a Artificial Intelligence Expert.
Transform this research paper abstract in triple backticks
into an executive summary for a healthcare company.
Have bullet points for pros and cons of ethics in Generative AI as mentioned in the paper.

- Act as a Artificial Intelligence Expert.
Transform this research paper abstract in triple backticks
into an executive summary for a generative AI company solving healthcare problems.
Have bullet points for key points mentioned for
Generative AI for text, images and structured data based healthcare

Use Conversation Chains with `ConversationBufferMemory` and modify ChatGPT behavior with System prompts as necessary to do the above 3

In [ ]:
paper_abstract = f"""
The widespread use of ChatGPT and other emerging technology powered by generative
artificial intelligence (AI) has drawn much attention to potential ethical issues, especially in
high-stakes applications such as healthcare.1–3 However, less clear is how to resolve such
issues beyond following guidelines and regulations that are still under discussion and
development. On the other hand, other types of generative AI have been used to synthesize
images and other types of data for research and practical purposes, which have resolved some
ethical issues and exposed other ethical issues,4,5 but such technology is less often the focus
of ongoing ethical discussions. Here we highlight gaps in current ethical discussions of
generative AI via a systematic scoping review of relevant existing research in healthcare, and
reduce the gaps by proposing an ethics checklist for comprehensive assessment and
transparent documentation of ethical discussions in generative AI development. While the
checklist can be readily integrated into the current peer review and publication system to
enhance generative AI research, it may also be used in broader settings to disclose ethicsrelated considerations in generative AI-powered products (or real-life applications of such
products) to help users establish reasonable trust in their capabilities.

Current ethical discussions on generative AI in healthcare
We conducted a systematic scoping review to analyse current ethical discussions on
generative AI in healthcare. Our search in four major academic research databases for
relevant publications from January 2013 to July 2023 yielded 2859 articles (see Methods for
detailed search strategy and Supplementary Figure S1 for the PRISMA flow diagram), of
which 193 articles were included for analysis based on application data modality (text, image,
or structured data), ethical issues discussed, generative AI involved, and whether generative
AI causes or offers technical solutions for issues raised.

Generative AI for text data-based healthcare
Forty-one of the 193 articles discussed ethical considerations pertaining to generative AI
applications for text data, with 20 articles describing methodological developments or
applications of generative AI and the other 21 articles describing review-type works on this
topic. Although some of these review-type articles used the general term “generative AI”, the
main body and supporting evidence focused on LLMs. Twenty-nine articles had in-depth
discussions on ethical issues, whereas the other 12 articles only briefly touched on some
ethical aspects.
Among the 41 articles, 29 articles focused on discussing ethical issues caused by LLMs (and
specifically by GPT in 16 of the articles), covering a wide range of application scenarios and
considered the application of all 10 ethical principles identified in the review (see Figure 1),
as well as other less discussed concerns such as human-AI interaction, and the rights of
LLMs to be considered as co-authors in scientific papers. One paper only commented briefly
on the need for ethical considerations in LLMs and is summarised in the “Others” category.
Although all ethical principles are equally important, some are discussed more often than
others, e.g., non-maleficence (also referred to in the literature as ‘benevolence’), equity, and
privacy.
Fifteen of the 41 articles aimed to resolve some existing ethical issues (for example,
confidentiality of medical data) by using LLMs and other generative AI (e.g., GAN,
autoencoder or diffusion), such as, to reduce privacy concerns by generating synthetic
medical text, to reduce disparity by providing accessible services and assistance, to detect
health-related misinformation, to generate trusted content, and to improve accountability or
transparency over existing approaches. While most articles focused on either identifying
ethical issues caused by generative AI or proposing generative AI-based solutions, three
articles discussed both to provide a more balanced perspective.

Generative AI for image and structured data-based healthcare
Unlike the diverse application scenarios of generative AI based on text data, for image and
structured data, this use of generative AI focuses on data synthesis and encryption. Hence the
majority of articles discussed the methodological developments of generative AI as giving
rise to a more distinctive and focused set of ethical issues.
5
Notably, of the 98 articles on image data and 58 articles on structured data, more than half
(n=63 for image data and n=33 for structured data) only mentioned ethical considerations as a
brief motivation for methodological developments or as a general discussion point. The rest
included more in-depth discussions or evaluations of ethical issues. Among these 155 articles
(as one article covered multiple modalities), 11 articles were review-type work, where 10
articles reviewed methods that mentioned one or two ethical perspectives, and only one
article24 discussed detailed ethical concerns on generative AI applications.
Resolving privacy issues was the main aim of articles for these two data modalities (n=74 for
image data and n=50 for structured data; see Figure 1), predominantly by generating synthetic
data using GAN. Eight articles on image data and 9 articles on structured data used
generative AI to reduce bias, e.g., by synthesizing data for under-represented subgroups in
existing databases. For both data modalities, we did not see explicit discussions on resolving
autonomy, integrity, or morality issues using generative AI, and for structured data the articles
additionally lacked discussions on trust or transparency.
Only 11 articles for image data selectively discussed some ethical issues that generative AI
can give rise to, without specific discussions regarding autonomy, integrity, or morality. For
structured data, only 4 articles discussed equity, privacy, or data security issues caused by
generative AI. Only two articles on structured data included both the cause and resolving
perspectives by discussing ethical issues that may arise from limitations of methods
proposed, specifically bias induced when synthesizing data in order to resolve privacy issues.
"""

In [ ]:
SYS_PROMPT = """
Act as a Artificial Intelligence Expert.
Transform the input research paper abstract in triple backticks
based on the audience input by the user.
"""
prompt =

memory =

conversation_chain =

In [ ]:
prompt = f"""
Transform this research paper abstract
into a short concise version of
maximum 10 lines for your audience.
Output summary should not have triple backticks.


Abstract:
```{paper_abstract}```
"""
user_instruction =
response =
print(response.content)

In [ ]:
# save conversation to memory
memory.save_context(user_instruction, {"output": response.content})

In [ ]:
prompt = f"""
Now build an executive summary for a healthcare company.
Have bullet points for pros and cons of ethics in Generative AI
as mentioned in the paper earlier.
"""

user_instruction =
response =
print(response.content)

In [ ]:
# save conversation to memory
memory.save_context(user_instruction, {"output": response.content})

In [ ]:
prompt = f"""
Now build an executive summary for a generative AI company solving healthcare problems.
Have bullet points for key points mentioned for
Generative AI for text, images and structured data based healthcare
"""

user_instruction =
response =
print(response.content)

## BONUS: Build a Conversational Chatbot

In [ ]:
# Import necessary components from the LangChain library.
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

def run_chatgpt_chatbot(system_prompt='', history_window=30, temperature=0.3):

  model = ChatOpenAI(model_name='gpt-3.5-turbo',
                     temperature=temperature)

  if system_prompt:
    SYS_PROMPT = system_prompt
  else:
    SYS_PROMPT = """
                  Act as a helpful AI Assistant
                 """

  prompt = ChatPromptTemplate.from_messages(
      [
          ("system", SYS_PROMPT),
          MessagesPlaceholder(variable_name="history"),
          ("human", "{input}"),
      ]
  )

  memory = ConversationBufferWindowMemory(k=history_window,
                                          return_messages=True)

  conversation_chain = (
      RunnablePassthrough.assign(
          history=RunnableLambda(memory.load_memory_variables)
          |
          itemgetter("history")
      )
      |
      prompt
      |
      model
  )

  # Print a welcome message when the chatbot starts.
  print("Hello! I am your friendly chatbot. Let's chat! (type 'STOP' to end)")

  # Start an infinite loop for interactive conversation with the user.
  while True:
    # Get input from the user.
    prompt = input('User: >>> ')
    # Check if the user wants to end the chat.
    if prompt.strip().upper() == 'STOP':
      print("ChatGPT: >>> Goodbye!")
      break

    # Generate and print the chatbot's reply.
    user_inp = {'input': prompt}
    reply = conversation_chain.invoke(user_inp)
    print(f"ChatGPT: >>>\n{reply.content}")
    memory.save_context(user_inp, {"output": reply.content})

In [ ]:
run_chatgpt_chatbot()

In [ ]:
run_chatgpt_chatbot(system_prompt='Act as a sarcastic child')